In [1]:
import h2o
import numpy as np
import math
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators import H2ORandomForestEstimator
from h2o.grid.grid_search import H2OGridSearch
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_121"; OpenJDK Runtime Environment (Zulu 8.20.0.5-linux64) (build 1.8.0_121-b15); OpenJDK 64-Bit Server VM (Zulu 8.20.0.5-linux64) (build 25.121-b15, mixed mode)
  Starting server from /home/rocket/anaconda2/h2o_jar/h2o.jar
  Ice root: /tmp/tmpcA_IWq
  JVM stdout: /tmp/tmpcA_IWq/h2o_rocket_started_from_python.out
  JVM stderr: /tmp/tmpcA_IWq/h2o_rocket_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster version:,3.10.0.9
H2O cluster version age:,"1 year, 2 months and 24 days !!!"
H2O cluster name:,H2O_from_python_rocket_5u3500
H2O cluster total nodes:,1
H2O cluster free memory:,3.464 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


In [2]:
df = h2o.import_file(path="../datasets/kickstarter-2018-reduced.csv")
df.describe()

Parse progress: |█████████████████████████████████████████████████████████| 100%
Rows:29999
Cols:10




,category,main_category,currency,goal,launched_utc,state,backers,country,usd pledged,time_open
type,enum,enum,enum,real,int,enum,int,enum,real,int
mins,0.0,0.0,0.0,1.0,3600.0,0.0,0.0,0.0,0.0,24.0
mean,NaN,NaN,NaN,39679.8923187,1411614526.6,NaN,104.256908564,NaN,6904.48511524,831.233841128
maxs,158.0,14.0,13.0,100000000.0,1514880718.0,5.0,85581.0,22.0,8596474.58,356809.0
sigma,NaN,NaN,NaN,743464.274157,61066107.2589,NaN,899.326567671,NaN,75466.6634392,2078.18963537
zeros,58,2266,622,0,0,3032,4275,51,5216,0
missing,0,0,0,0,0,0,0,0,304,0
0,Poetry,Publishing,GBP,1000.0,1439295148.0,failed,0.0,GB,0.0,1415.0
1,Narrative Film,Film & Video,USD,30000.0,1504327437.0,failed,15.0,US,100.0,1438.0
2,Narrative Film,Film & Video,USD,45000.0,1357950050.0,failed,3.0,US,220.0,1080.0


In [3]:
# Define predictors
y = df['state']
X = df
X = X.drop(['state', 'backers', 'country', 'usd pledged'])
X.describe()
y.describe()

Rows:29999
Cols:6




,category,main_category,currency,goal,launched_utc,time_open
type,enum,enum,enum,real,int,int
mins,0.0,0.0,0.0,1.0,3600.0,24.0
mean,NaN,NaN,NaN,39679.8923187,1411614526.6,831.233841128
maxs,158.0,14.0,13.0,100000000.0,1514880718.0,356809.0
sigma,NaN,NaN,NaN,743464.274157,61066107.2589,2078.18963537
zeros,58,2266,622,0,0,0
missing,0,0,0,0,0,0
0,Poetry,Publishing,GBP,1000.0,1439295148.0,1415.0
1,Narrative Film,Film & Video,USD,30000.0,1504327437.0,1438.0
2,Narrative Film,Film & Video,USD,45000.0,1357950050.0,1080.0


Rows:29999
Cols:1




,state
type,enum
mins,0.0
mean,NaN
maxs,5.0
sigma,NaN
zeros,3032
missing,0
0,failed
1,failed
2,failed


In [4]:
# Split for training
train, valid, test = df.split_frame(
    ratios = [0.6, 0.2],
    seed=4567,
    destination_frames=['train.hex','valid.hex','test.hex']
)

In [5]:
# Single decision tree, no boosting single tree
rfe = H2ORandomForestEstimator(ntrees=1)
rfe.train(x=X.columns, y=y.columns[0], training_frame=train)

# Show model summary
print rfe

drf Model Build progress: |███████████████████████████████████████████████| 100%
Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_model_python_1517160836387_1
Model Summary: 


,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,1.0,6.0,68107.0,11.0,20.0,17.666666,47.0,1936.0,774.5




ModelMetricsMultinomial: drf
** Reported on train data. **

MSE: 0.531895812537
RMSE: 0.729311876043
LogLoss: 15.7755368063
Mean Per-Class Error: 0.589138242633
Confusion Matrix: vertical: actual; across: predicted



canceled,failed,live,successful,suspended,undefined,Error,Rate
72.0,331.0,0.0,265.0,0.0,0.0,0.8922156,596 / 668
286.0,1817.0,5.0,1334.0,11.0,3.0,0.4742477,"1,639 / 3,456"
6.0,4.0,27.0,10.0,0.0,0.0,0.4255319,20 / 47
167.0,900.0,0.0,1294.0,4.0,8.0,0.4546987,"1,079 / 2,373"
4.0,10.0,0.0,10.0,0.0,0.0,1.0,24 / 24
4.0,8.0,0.0,4.0,1.0,42.0,0.2881356,17 / 59
539.0,3070.0,32.0,2917.0,16.0,53.0,0.5092802,"3,375 / 6,627"


Top-6 Hit Ratios: 


k,hit_ratio
1,0.4907198
2,0.833107
3,0.8681153
4,0.8687189
5,0.8687189
6,1.0


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error
,2018-01-28 12:33:59,0.016 sec,0.0,nan,nan,nan
,2018-01-28 12:34:00,0.835 sec,1.0,0.7293119,15.7755368,0.5092802


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
category,1603.7889404,1.0,0.4090965
launched_utc,746.4356689,0.4654201,0.1904017
goal,572.1552124,0.3567522,0.1459461
time_open,470.0491638,0.2930867,0.1199007
main_category,300.5323486,0.1873890,0.0766602
currency,227.3581543,0.1417631,0.0579948


In [6]:
perf = rfe.model_performance(valid)
perf


ModelMetricsMultinomial: drf
** Reported on test data. **

MSE: 0.532875994861
RMSE: 0.729983557939
LogLoss: 15.6708229318
Mean Per-Class Error: 0.597857725129
Confusion Matrix: vertical: actual; across: predicted



canceled,failed,live,successful,suspended,undefined,Error,Rate
78.0,399.0,6.0,164.0,4.0,1.0,0.8803681,574 / 652
315.0,1883.0,12.0,885.0,12.0,13.0,0.3964744,"1,237 / 3,120"
9.0,10.0,31.0,1.0,1.0,0.0,0.4038462,21 / 52
188.0,998.0,3.0,899.0,8.0,10.0,0.5731244,"1,207 / 2,106"
2.0,19.0,1.0,8.0,0.0,0.0,1.0,30 / 30
0.0,7.0,0.0,10.0,0.0,34.0,0.3333333,17 / 51
592.0,3316.0,53.0,1967.0,25.0,58.0,0.5133921,"3,086 / 6,011"


Top-6 Hit Ratios: 


k,hit_ratio
1,0.4866079
2,0.7927133
3,0.8303111
4,0.8306438
5,0.8306438
6,0.9999999


In [7]:
# Lets see if we can do better on one tree
hyper_parameters = {'ntrees':[1], 
                    'max_depth':[20, 5], 
                    'balance_classes':[True, False], 
                    'distribution':['bernoulli', 'multinominal', 'gaussian', 'poisson', 'gamma', 'tweedie', 'laplace', 'huber'],
                    'histogram_type':['uniform_adaptive', 'random', 'quantiles_global', 'round_robin']}
grid_search = H2OGridSearch(H2ORandomForestEstimator, hyper_params=hyper_parameters)
grid_search.train(x=X.columns, y=y.columns[0], training_frame=train)
grid_search.show()

drf Grid Build progress: |████████████████████████████████████████████████| 100%
       balance_classes distribution   histogram_type max_depth ntrees  \
0                false      laplace  QuantilesGlobal         5      1   
1                false        huber       RoundRobin         5      1   
2                false        gamma           Random         5      1   
3                false        gamma  QuantilesGlobal         5      1   
4                false    bernoulli  QuantilesGlobal         5      1   
5                false        gamma       RoundRobin         5      1   
6                false    bernoulli  UniformAdaptive         5      1   
7                false      tweedie  QuantilesGlobal         5      1   
8                false      poisson  UniformAdaptive         5      1   
9                false      poisson       RoundRobin         5      1   
10               false        gamma  UniformAdaptive         5      1   
11               false     gaussian       R

[112 rows x 8 columns]


In [8]:
# Now select the configuration with the lowest logloss
opt_tree = grid_search.get_grid(sort_by='logloss', decreasing=True)[-1]

In [9]:
opt_tree.model_performance(valid)


ModelMetricsMultinomial: drf
** Reported on test data. **

MSE: 0.33251391491
RMSE: 0.576640195365
LogLoss: 1.14402647794
Mean Per-Class Error: 0.760539927537
Confusion Matrix: vertical: actual; across: predicted



canceled,failed,live,successful,suspended,undefined,Error,Rate
2.0,517.0,2.0,130.0,1.0,0.0,0.9969325,650 / 652
6.0,2497.0,4.0,612.0,1.0,0.0,0.1996795,"623 / 3,120"
1.0,36.0,6.0,9.0,0.0,0.0,0.8846154,46 / 52
3.0,1130.0,2.0,967.0,3.0,1.0,0.5408357,"1,139 / 2,106"
0.0,28.0,0.0,2.0,0.0,0.0,1.0,30 / 30
1.0,26.0,0.0,21.0,0.0,3.0,0.9411765,48 / 51
13.0,4234.0,14.0,1741.0,5.0,4.0,0.4218932,"2,536 / 6,011"


Top-6 Hit Ratios: 


k,hit_ratio
1,0.5781068
2,0.8704042
3,0.9820329
4,0.9933455
5,0.9945101
6,1.0


In [10]:
h2o.shutdown()

H2O session _sid_ade8 closed.
